In [1]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import os

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

import xgboost
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.wrappers.scikit_learn import KerasClassifier
# fix random seed for reproducibility
np.random.seed(26)

/Users/LF/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
train = pd.read_csv('bases/training_variants')
test = pd.read_csv('bases/test_variants')
train_texts = pd.read_csv('bases/training_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")
test_texts = pd.read_csv('bases/test_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")

In [3]:
train = pd.merge(train, train_texts, how='left', on='ID')
test = pd.merge(test, test_texts, how='left', on='ID')

In [4]:
#### process the train and test set together
data_all = pd.concat((train, test), axis=0, ignore_index=True)

In [5]:
stop = set(stopwords.words('english'))
exclude = set('!"#$%&\'()*+:;<=>?@[\\]^_`{|}~0123456789') 
lemma = WordNetLemmatizer()
def clean(doc,lemmatiz=False):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free_0 = [re.sub(",|\.|/"," ",ch) for ch in stop_free]
    if lemmatiz:
        punc_free_lem="".join(ch for ch in punc_free_0 if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free_lem.split())
        return normalized
    else:
        punc_free = "".join(ch for ch in punc_free_0 if ch not in exclude)
        return punc_free

In [8]:
#No lemmatization for the moment, be careful not to lemmatize then w2vec
data_all.Text = [clean(doc,lemmatiz=True) for doc in data_all.Text]  

In [9]:
ID_train=train.ID
ID_test=test.ID

In [10]:
train = data_all.iloc[:len(train)]
test = data_all.iloc[len(train):]

In [27]:
y=train["Class"]-1
X_train=train.drop(["Class","ID","Gene","Variation"],axis=1)
X_test=test.drop(["Class","ID","Gene","Variation"],axis=1)
txt_no_dup=train["Text"].drop_duplicates()

In [12]:
tfidf = TfidfVectorizer(
        min_df=10, max_features=15000, strip_accents=None, lowercase = False,
        analyzer='word', token_pattern=r'\w+', ngram_range=(1,3), use_idf=True,
        smooth_idf=True, sublinear_tf=True
        ).fit(txt_no_dup)

In [13]:
X_train_text = tfidf.transform(train["Text"])
X_test_text = tfidf.transform(test["Text"])

In [14]:
list_comp=[10,20,30,40,50,60,80,100,120,150,170,200]
dic_svd={}
for comp in list_comp:
    dic_svd[comp]=TruncatedSVD(n_components=comp,n_iter=10,random_state=26)
tsvd_train,tsvd_test = {},{}
for sv in dic_svd:
    tsvd_train[sv]=dic_svd[sv].fit_transform(X_train_text)
    tsvd_test[sv]=dic_svd[sv].transform(X_test_text)

In [15]:
for n in dic_svd:
    for i in range(n):
        X_train['tsvd_' +str(n)+"_"+str(i)] = tsvd_train[n][:, i]
        X_test['tsvd_' +str(n)+"_"+str(i)] = tsvd_test[sv][:, i]

In [16]:
X_train_0=np.array(X_train.drop("Text",axis=1))
X_test_0=np.array(X_test.drop("Text",axis=1))

In [ ]:
#LSTM classifier

In [17]:
# create the model
def model_ann():
    model = Sequential()
    model.add(Dense(300,input_dim=1030,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(300,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(300,activation="relu"))
    model.add(Dense(9,activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [19]:
dic_classifier={"XGB_medium":XGBClassifier(n_estimators=200,max_depth=5, objective="multi:softprob",subsample=0.7,seed=26),
    "XGB_small":XGBClassifier(max_depth=2,objective="multi:softprob",subsample=0.5,seed=26),
                   "XGB_tall":XGBClassifier(n_estimators=300,max_depth=7,subsample=0.9,objective="multi:softprob",seed=26)}

In [20]:
def model_gen_nowdw(X,X_test,y,classifier,file,neural_net=False):

    kf = StratifiedKFold(n_splits=5, random_state=26, shuffle=True)
    if neural_net==False:
        fold = 0
        y_test=0
        for train_index, test_index in kf.split(X, y):
            fold += 1
            X_train, X_valid    = X[train_index],   X[test_index]
            y_train, y_valid    = y[train_index],   y[test_index]

            print("Fold", fold, X_train.shape, X_valid.shape)
            clf=classifier
            clf.fit(X_train,y_train)
            y_test = clf.predict_proba(X_test)
    else:
        print("One Fold predict for NN")
        clf=classifier
        clf.fit(X,y)
        y_test=clf.predict_proba(X_test)
    classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
    subm = pd.DataFrame(y_test, columns=classes)
    subm['ID'] = ID_test
    
    subm.to_csv("scores/stack_test/nowdw_{}_new.csv".format(file),index=False)
    
    print("cross_val sur train") #peut etre que to array est exclusivement pour les xgb
    
    if os.path.isfile("scores/stack_train/nowdw_{}_new.csv".format(file)):
        print("not necessary, already done")
    else:
        if neural_net==False:
            y_pred=cross_val_predict(estimator=clf,X=X,y=y,cv=kf,method="predict_proba")
        else:
            y_pred=clf.predict_proba(X)
        subm1 = pd.DataFrame(y_pred, columns=classes)
        subm1['ID'] = ID_train
        subm1.to_csv("scores/stack_train/nowdw_{}_new.csv".format(file),index=False)


In [20]:
for clf in dic_classifier:
    model_gen_nowdw(X_train_0,X_test_0,y,dic_classifier[clf],file=clf)

Fold 1 (2653, 1030) (668, 1030)
Fold 2 (2654, 1030) (667, 1030)
Fold 3 (2657, 1030) (664, 1030)
Fold 4 (2659, 1030) (662, 1030)
Fold 5 (2661, 1030) (660, 1030)
cross_val sur train
Fold 1 (2653, 1030) (668, 1030)
Fold 2 (2654, 1030) (667, 1030)
Fold 3 (2657, 1030) (664, 1030)
Fold 4 (2659, 1030) (662, 1030)
Fold 5 (2661, 1030) (660, 1030)
cross_val sur train
Fold 1 (2653, 1030) (668, 1030)
Fold 2 (2654, 1030) (667, 1030)
Fold 3 (2657, 1030) (664, 1030)
Fold 4 (2659, 1030) (662, 1030)
Fold 5 (2661, 1030) (660, 1030)
cross_val sur train


# REPEATABLE NEURAL NETWORK

In [50]:
from keras.callbacks import EarlyStopping,ModelCheckpoint

In [130]:
X_train_0.shape

(3321, 1030)

In [132]:
def fit_model_nn(X, y):
    model = Sequential()
    model.add(Dense(300,input_dim=1030,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(300,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(300,activation="relu"))
    model.add(Dense(9,activation="softmax"))
    epochs=200
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    es=EarlyStopping(monitor="loss",mode="min",patience=10)
    checkpoint=ModelCheckpoint(filepath="checkann",save_best_only=True,monitor="loss",mode="min")
    model.fit(X, y,epochs=100,batch_size=50,callbacks=[es,checkpoint],validation_split=0.2)
    model.load_weights("checkann")
    y_test=model.predict_proba(X_test_0)
    return y_test

In [133]:
y_cat=to_categorical(y,9)

In [111]:
# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=26)
cvscores = []
for train, test in kfold.split(X_train_0, y):
  # create model
    model = Sequential()
    model.add(Dense(100, input_dim=1030, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(300,activation="relu"))
    model.add(Dense(9, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    es=EarlyStopping(monitor="loss",mode="min",patience=10)
    checkpoint=ModelCheckpoint(filepath="checknn",save_best_only=True,monitor="loss",mode="min")
    model.fit(X_train_0[train], y_cat[train],epochs=100,batch_size=50,validation_split=0.2,callbacks=[es,checkpoint],verbose=0)
    model.load_weights("checknn")
    # evaluate the model
    scores = model.evaluate(X_train_0[test], y_cat[test])
    print("%s: %.2f" % (model.metrics_names[0], scores[0]))
    cvscores.append(scores[0] )
print("%.2f (+/- %.2f)" % (np.mean(cvscores), np.std(cvscores)))

384/660 [================>.............] - ETA: 0sloss: 2.59
2.22 (+/- 0.25)


In [124]:
classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
subm = pd.DataFrame(y_sol, columns=classes)
subm['ID'] = ID_test
subm.to_csv("scores/stack_test/neural_net_overfitting.csv",index=False)

In [ ]:
#repeating
repeats=10
scores={}
for i in range(repeats):
    scores["rep_"+str(i)]=fit_model_nn(X_train_0,to_categorical(y,9))

Train on 2656 samples, validate on 665 samples
Epoch 1/100
2656/2656 [==============================] - 6s - loss: 1.4908 - acc: 0.4974 - val_loss: 1.5952 - val_acc: 0.3985
Epoch 2/100
2656/2656 [==============================] - 1s - loss: 1.0469 - acc: 0.6096 - val_loss: 1.5901 - val_acc: 0.3744
Epoch 3/100
2656/2656 [==============================] - 1s - loss: 0.9519 - acc: 0.6434 - val_loss: 1.4809 - val_acc: 0.3985
Epoch 4/100
2656/2656 [==============================] - 1s - loss: 0.8743 - acc: 0.6642 - val_loss: 1.5278 - val_acc: 0.4827
Epoch 5/100
2656/2656 [==============================] - 1s - loss: 0.8390 - acc: 0.6709 - val_loss: 1.6329 - val_acc: 0.3955
Epoch 6/100
2656/2656 [==============================] - 1s - loss: 0.7931 - acc: 0.6875 - val_loss: 1.6695 - val_acc: 0.4707
Epoch 7/100
2656/2656 [==============================] - 1s - loss: 0.7612 - acc: 0.6928 - val_loss: 1.7883 - val_acc: 0.4000
Epoch 8/100
2656/2656 [==============================] - 1s - loss: 0.7

2656/2656 [==============================] - 0s - loss: 0.3440 - acc: 0.8404 - val_loss: 4.0004 - val_acc: 0.4045
Epoch 66/100
2656/2656 [==============================] - 0s - loss: 0.3307 - acc: 0.8486 - val_loss: 4.1251 - val_acc: 0.4165
Epoch 67/100
2656/2656 [==============================] - 0s - loss: 0.3335 - acc: 0.8419 - val_loss: 4.2194 - val_acc: 0.3880
Epoch 68/100
2656/2656 [==============================] - 0s - loss: 0.3331 - acc: 0.8441 - val_loss: 4.4825 - val_acc: 0.3699
Epoch 69/100
2656/2656 [==============================] - 0s - loss: 0.3357 - acc: 0.8370 - val_loss: 4.2547 - val_acc: 0.3850
Epoch 70/100
2656/2656 [==============================] - 0s - loss: 0.3353 - acc: 0.8392 - val_loss: 4.3336 - val_acc: 0.3744
Epoch 71/100
2656/2656 [==============================] - 0s - loss: 0.3347 - acc: 0.8441 - val_loss: 4.3449 - val_acc: 0.3624
Epoch 72/100
2656/2656 [==============================] - 0s - loss: 0.3378 - acc: 0.8502 - val_loss: 4.3508 - val_acc: 0.38

2656/2656 [==============================] - 1s - loss: 0.4423 - acc: 0.8016 - val_loss: 3.0395 - val_acc: 0.4406
Epoch 29/100
2656/2656 [==============================] - 1s - loss: 0.4362 - acc: 0.8136 - val_loss: 2.8719 - val_acc: 0.4692
Epoch 30/100
2656/2656 [==============================] - 1s - loss: 0.4153 - acc: 0.8193 - val_loss: 3.0527 - val_acc: 0.4827
Epoch 31/100
2656/2656 [==============================] - 1s - loss: 0.4176 - acc: 0.8219 - val_loss: 3.1800 - val_acc: 0.4090
Epoch 32/100
2656/2656 [==============================] - 1s - loss: 0.4066 - acc: 0.8170 - val_loss: 3.2152 - val_acc: 0.4030
Epoch 33/100
2656/2656 [==============================] - 1s - loss: 0.4044 - acc: 0.8170 - val_loss: 3.0635 - val_acc: 0.4602
Epoch 34/100
2656/2656 [==============================] - 1s - loss: 0.4026 - acc: 0.8219 - val_loss: 3.1820 - val_acc: 0.4451
Epoch 35/100
2656/2656 [==============================] - 1s - loss: 0.3864 - acc: 0.8223 - val_loss: 3.7763 - val_acc: 0.43

2656/2656 [==============================] - 1s - loss: 0.3116 - acc: 0.8524 - val_loss: 4.7008 - val_acc: 0.4030
Epoch 93/100
2656/2656 [==============================] - 1s - loss: 0.3095 - acc: 0.8543 - val_loss: 4.9429 - val_acc: 0.3353
Epoch 94/100
2656/2656 [==============================] - 0s - loss: 0.2983 - acc: 0.8528 - val_loss: 4.8547 - val_acc: 0.3308
Epoch 95/100
2656/2656 [==============================] - 0s - loss: 0.3052 - acc: 0.8528 - val_loss: 4.4439 - val_acc: 0.3684
Epoch 96/100
2656/2656 [==============================] - 0s - loss: 0.3190 - acc: 0.8490 - val_loss: 4.6787 - val_acc: 0.4045
Epoch 97/100
2656/2656 [==============================] - 0s - loss: 0.3165 - acc: 0.8498 - val_loss: 4.5144 - val_acc: 0.3579
Epoch 98/100
2656/2656 [==============================] - 0s - loss: 0.3146 - acc: 0.8494 - val_loss: 5.0220 - val_acc: 0.3383
Epoch 99/100
2656/2656 [==============================] - 0s - loss: 0.3061 - acc: 0.8581 - val_loss: 5.2007 - val_acc: 0.32

2656/2656 [==============================] - 1s - loss: 0.3501 - acc: 0.8422 - val_loss: 4.2103 - val_acc: 0.3504
Epoch 56/100
2656/2656 [==============================] - 1s - loss: 0.3398 - acc: 0.8419 - val_loss: 4.4988 - val_acc: 0.3549
Epoch 57/100
2656/2656 [==============================] - 0s - loss: 0.3472 - acc: 0.8381 - val_loss: 4.4164 - val_acc: 0.3910
Epoch 58/100
2656/2656 [==============================] - 0s - loss: 0.3468 - acc: 0.8385 - val_loss: 4.5293 - val_acc: 0.3624
Epoch 59/100
2656/2656 [==============================] - 1s - loss: 0.3431 - acc: 0.8392 - val_loss: 4.7596 - val_acc: 0.3459
Epoch 60/100
2656/2656 [==============================] - 1s - loss: 0.3368 - acc: 0.8468 - val_loss: 4.8281 - val_acc: 0.3444
Epoch 61/100
 650/2656 [======>.......................] - ETA: 0s - loss: 0.3199 - acc: 0.8477